## Running Backs

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')
from fantasy_football import Fantasy

# Import fantasy class which will be used for building datasets
football = Fantasy()

First, we pull 2022 offense data and clean it

In [3]:
df, y = Fantasy.read_data()

Split data by positional group

In [4]:
RB_data = football.makeRegularData(df, "RB")


Next, we pull more advanced data for rushing and passing

In [5]:
# Passing advanced
rushing_advanced = football.getAdvancedStats("rushing", "2021")

Now we are ready to merge the datasets to incorporate the more advance statistics

In [31]:
RB = football.addAdvanced(RB_data, rushing_advanced)

,Player,Pos_x,Team,GP,Att_x,Rushing_Yds,Rushing_Td,Rushing_1st,Tgt,Rec,...,Att_y,Yds,TD,1D,YBC,YBC/Att,YAC,YAC/Att,BrkTkl,Att/Br
0,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,332,1811,18,107,870,2.6,941,2.8,25,13.3
1,J. Taylor,RB,Ind,17,332,1811,18,107,51,40,...,19,37,2,3,25,1.3,12,0.6,0,
2,A. Ekeler,RB,LAC,16,206,911,12,53,94,70,...,206,911,12,53,477,2.3,434,2.1,13,15.8
3,J. Mixon,RB,Cin,16,292,1205,13,60,48,42,...,292,1205,13,60,662,2.3,543,1.9,20,14.6
4,N. Harris,RB,Pit,17,307,1200,7,62,94,74,...,307,1200,7,62,525,1.7,675,2.2,30,10.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
125,M. Burton,RB,KC,16,8,26,1,8,3,3,...,8,26,1,8,11,1.4,15,1.9,0,
126,C. Ham,RB,Min,17,7,34,0,3,18,17,...,7,34,0,3,23,3.3,11,1.6,0,
127,A. Walter,RB,LV,4,26,101,1,5,3,2,...,26,101,1,5,68,2.6,33,1.3,0,
128,P. Taylor,RB,GB,9,23,89,1,5,2,2,...,23,89,1,5,53,2.3,36,1.6,2,11.5


### CONTRACTS AND OTHER DATA (not yet completed)

We are not using the player contracts yet but here is where we scrape that data

In [56]:
table_contracts = pd.read_html('https://www.spotrac.com/nfl/contracts/sort-value/offense/all-time/free-agent-2021/limit-100/')

player_contracts = table_contracts[0]
player_contracts

,Rank,Player,Signed Age,Yrs,Value,AAV,Sign Bonus,G'teed @ Sign,Practical G'teed
0,1,Jay Cutler Quarterback | 2014-2020 (FA: 2021)...,30,7,"$126,700,000","$18,100,000",NaN,"$38,000,000","$54,000,000"
1,2,Cam Newton Quarterback | 2015-2020 (FA: 2021),26,5,"$103,800,000","$20,760,000","$22,500,000","$41,000,000","$60,000,000"
2,3,Andy Dalton Quarterback | 2014-2020 (FA: 2021...,26,6,"$96,000,000","$16,000,000","$12,000,000","$17,000,000","$17,000,000"
3,4,Kirk Cousins Quarterback | 2018-2020 (FA: 2021),29,3,"$84,000,000","$28,000,000","$3,000,000","$84,000,000","$84,000,000"
4,5,Ryan Tannehill Quarterback | 2015-2020 (FA: 2...,26,4,"$77,000,000","$19,250,000","$11,500,000","$21,500,000","$45,000,000"
...,...,...,...,...,...,...,...,...,...
95,96,Paxton Lynch Quarterback | 2016-2020 (FA: 202...,22,4,"$9,476,296","$2,369,074","$5,091,852","$7,884,074","$7,884,074"
96,97,Lee Smith Tight End | 2018-2020 (FA: 2021) O...,30,3,"$9,000,000","$3,000,000",NaN,"$3,000,000","$3,000,000"
97,97,Sammy Watkins Wide Receiver | 2020-2020 (FA: ...,26,1,"$9,000,000","$9,000,000","$1,000,000","$8,000,000","$8,000,000"
98,97,David Andrews Center | 2017-2020 (FA: 2021),24,3,"$9,000,000","$3,000,000","$1,600,000","$1,600,000","$1,600,000"
